In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path = "medicalai/MedFound-Llama3-8B-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

In [2]:
model.device

device(type='cuda', index=0)

In [6]:
input_text = f"""基于案例回答问题，单选题，从中选择一个最佳答案，要求只输出选项，不输出解释：
患者男性，25岁，足球运动员，在一次比赛中突然感到膝关节剧痛，随后肿胀明显，查体发现膝关节前抽屉试验阳性。最可能的诊断是：
A：侧副韧带损伤，B：前交叉韧带损伤，C：后交叉韧带断裂，D：髌骨脱位，E：半月板损伤"""
input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=False).to(model.device)
output_ids = model.generate(input_ids, max_new_tokens=10, temperature=1, do_sample=True).to(model.device)
generated_text = tokenizer.decode(output_ids[0,len(input_ids[0]):], skip_special_tokens=True)
print("Generated Output:\n", generated_text)


Generated Output:
 。
答案：选择C，后交叉


In [ ]:
prompt = f"""请根据患者的病史、体格检查、影像学检查结果，回答问题：
基本情况：46岁中年女性
病史：5+年前，患者无明显诱因出现右膝疼痛，以外侧为主，行走或运动后疼痛加重，休息后缓解，偶有弹响及卡锁感，否认发热寒战、肢体麻木等症状，采取针灸、抽取关节液和口服止痛药等对症治疗，症状无明显缓解。
体格检查：视：右膝无明显畸形，无皮肤瘀斑，无红肿，无破溃等。
触：右膝关节皮温不高，右膝关节外侧间隙压痛（+），足背动脉可扪及，双下肢感觉对称，无感觉减退。
动：右膝活动度：5°-130°，浮髌试验（-），外侧Mcmurray征（-），髌骨研磨试验（-），前抽屉试验（-），后抽屉试验（-），Lachman试验（-），外翻应力试验（-），内翻应力试验（-）。
量：双下肢基本等长。
MRI右侧膝关节普通扫描：右膝关节对位良好，踝间嵴、关节边缘骨质增生。膝关节软骨变薄，表面欠光滑。内侧半月板后角见线状长T2信号，延及关节面；外侧半月板前角增大，体部及前角见线状、片状长T2信号，延及关节面。前交叉韧带外份局部纤维走行欠规则，内见长T2信号；外侧副韧带上端稍增粗，内见稍长T2信号。后交叉韧带、内侧副韧带局部T2WI信号增高。膝关节腔少量积液，髌上囊中量积液，髌上囊内滑膜增多。膝关节周围软组织轻度肿胀。
请基于提供的病史、体格检查和影像学检查结果，进行全面分析，并输出针对该患者的个性化且符合循证医学原则的治疗建议。"""
messages = [{"role": "user", "content": prompt}]
# text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
output_ids = model.generate(**input_ids, max_new_tokens=2048).to(model.device)
generated_text = tokenizer.decode(output_ids[0,len(input_ids[0]):], skip_special_tokens=True)
print("Generated Output:\n", generated_text)

Generated Output:
  I am sorry to read that you are experiencing difficulty sleeping. I understand that you are taking Ambien to help you sleep, but I am concerned that you are taking too much of it. I would recommend that you reduce your Ambien dose or try another sleep aid. I would also recommend that you try to get more sleep during the day. If you are still experiencing difficulty sleeping, I would recommend that you consult with your doctor about other options for sleep aids. I am glad that you are taking the Ambien to help you sleep, but I would like to make sure that you are not taking too much of it. I would also recommend that you try to get more sleep during the day. If you are still experiencing difficulty sleeping, I would recommend that you consult with your doctor about other options for sleep aids. I am glad that you are taking the Ambien to help you sleep, but I would like to make sure that you are not taking too much of it. I would also recommend that you try to get mo

In [7]:
from tqdm import tqdm
import re
def prediction(excel_path):
  sheets = pd.read_excel(excel_path, sheet_name=None)

    # 2. 用 ExcelWriter 打开同一个文件，替换写入每个 sheet
  with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
      for sheet_name, data in sheets.items():
        data["predict_medfound_7b"] = ""
        for i in tqdm(range(len(data)), desc=f"Processing {sheet_name}"):
          query = data.loc[i, "question"]
          input_ids = tokenizer.encode(query, return_tensors="pt", add_special_tokens=False).to(model.device)
          output_ids = model.generate(input_ids, max_new_tokens=20, temperature=0.1, do_sample=True).to(model.device)
          generated_text = tokenizer.decode(output_ids[0,len(input_ids[0]):], skip_special_tokens=True)
          m = re.search(r"([A-Z])(：(.*?))?\s<\|endoftext\|>", generated_text)
          if m:
              answer = m.group(1)
          else:
              answer = generated_text.strip()
          data.loc[i, "predict_medfound_7b"] = answer
      data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例分析治疗题-报告版.xlsx")

Processing 治疗: 100%|██████████| 206/206 [4:28:23<00:00, 78.17s/it]


In [ ]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例分析诊断题-报告版.xlsx")

Processing 诊断: 100%|██████████| 206/206 [4:27:19<00:00, 77.86s/it]


In [ ]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例文本单选题.xlsx")

Processing Sheet1: 100%|██████████| 101/101 [2:10:28<00:00, 77.51s/it]


In [8]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/简单文本单选题.xlsx")

Processing Sheet1: 100%|██████████| 129/129 [01:49<00:00,  1.18it/s]


In [6]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.novita.ai/openai",
    api_key="",
)

model = "baichuan/baichuan-m2-32b"
stream = False  # or False
max_tokens = 2048


from tqdm import tqdm
import re
import pandas as pd
def prediction(excel_path):
  sheets = pd.read_excel(excel_path, sheet_name=None)

    # 2. 用 ExcelWriter 打开同一个文件，替换写入每个 sheet
  with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
      for sheet_name, data in sheets.items():
        data["predict_baichuan-m2-32b"] = ""
        for i in tqdm(range(len(data)), desc=f"Processing {sheet_name}"):
          query = data.loc[i, "question"]
          completion_res = client.completions.create(
                model=model,
                prompt=query,
                stream=stream,
                max_tokens=max_tokens,
            )
          generated_text = completion_res.choices[0].text
          m = re.search(r"([A-Z])(：(.*?))?\s<\|endoftext\|>", generated_text)
          if m:
              answer = m.group(1)
          else:
              answer = generated_text.strip()
          data.loc[i, "predict_baichuan-m2-32b"] = answer
      data.to_excel(writer, sheet_name=sheet_name, index=False)

In [7]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例分析治疗题-报告版.xlsx")

Processing 治疗: 100%|██████████| 206/206 [2:33:00<00:00, 44.56s/it]


In [8]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例分析诊断题-报告版.xlsx")

Processing 诊断: 100%|██████████| 206/206 [2:15:34<00:00, 39.49s/it]


In [9]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/病例文本单选题.xlsx")

Processing Sheet1: 100%|██████████| 101/101 [1:08:29<00:00, 40.69s/it]


In [10]:
prediction("/content/drive/MyDrive/医疗数据集/second_milistone/简单文本单选题.xlsx")

Processing Sheet1: 100%|██████████| 129/129 [1:29:08<00:00, 41.46s/it]
